In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="coco8.yaml", epochs=3)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg")

# Export the model to ONNX format
success = model.export(format="onnx")


In [ ]:
!pip install ultralytics
from ultralytics import YOLO
# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(["/content/bus.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

In [ ]:
import cv2

from ultralytics import solutions

cap = cv2.VideoCapture("path/to/video.mp4")
assert cap.isOpened(), "Error reading video file"

# Video writer
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("workouts_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init AIGym
gym = solutions.AIGym(
    show=True,  # display the frame
    kpts=[6, 8, 10],  # keypoints for monitoring specific exercise, by default it's for pushup
    model="yolo11n-pose.pt",  # path to the YOLO11 pose estimation model file
    # line_width=2,  # adjust the line width for bounding boxes and text display
)

# Process video
while cap.isOpened():
    success, im0 = cap.read()

    if not success:
        print("Video frame is empty or processing is complete.")
        break

    results = gym(im0)

    # print(results)  # access the output

    video_writer.write(results.plot_im)  # write the processed frame.

cap.release()
video_writer.release()
cv2.destroyAllWindows()  # destroy all opened windows

In [ ]:
import cv2

from ultralytics import solutions

# Video capture
cap = cv2.VideoCapture("path/to/video.mp4")
assert cap.isOpened(), "Error reading video file"

# Video writer
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("parking management.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize parking management object
parkingmanager = solutions.ParkingManagement(
    model="yolo11n.pt",  # path to model file
    json_file="bounding_boxes.json",  # path to parking annotations file
)

while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break

    results = parkingmanager(im0)

    # print(results)  # access the output

    video_writer.write(results.plot_im)  # write the processed frame.

cap.release()
video_writer.release()
cv2.destroyAllWindows()  # destroy all opened windows

In [ ]:
!uv pip install ultralytics
import ultralytics

ultralytics.checks()


In [ ]:
!pip install ultralytics

# !yolo track source="/path/to/video.mp4" save=True
from collections import defaultdict

import cv2

from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

# Dictionary to store tracking history with default empty lists
track_history = defaultdict(lambda: [])

# Load the YOLO model with segmentation capabilities
model = YOLO("yolo11n-seg.pt")

# Open the video file
cap = cv2.VideoCapture("sample.mp4")

# Retrieve video properties: width, height, and frames per second
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Initialize video writer to save the output video with the specified properties
out = cv2.VideoWriter("instance-segmentation-object-tracking.avi", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h))

while True:
    # Read a frame from the video
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Create an annotator object to draw on the frame
    annotator = Annotator(im0, line_width=2)

    # Perform object tracking on the current frame
    results = model.track(im0, persist=True)

    # Check if tracking IDs and masks are present in the results
    if results[0].boxes.is_track and results[0].masks is not None:
        # Extract masks and tracking IDs
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Annotate each mask with its corresponding tracking ID and color
        for mask, track_id in zip(masks, track_ids):
            if mask.size > 0: # Check if the mask is not empty
                annotator.seg_masks(masks=[mask], ids=[track_id], ret_native=True) # Annotate each mask individually

    # Write the annotated frame to the output video
    out.write(im0)
    # Display the annotated frame
    # cv2.imshow("instance-segmentation-object-tracking", im0)

    # Exit the loop if 'q' is pressed
    # if cv2.waitKey(1) & 0xFF == ord("q"):
    #     break

# Release the video writer and capture objects, and close all OpenCV windows
out.release()
cap.release()
# cv2.destroyAllWindows()

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
import time
from ultralytics import YOLO
from ultralytics.utils.plotting import colors
from google.colab.patches import cv2_imshow # Import cv2_imshow

# Dictionary to store tracking history (optional for trails)
track_history = defaultdict(lambda: [])

# Load YOLOv8 segmentation model
model = YOLO("yolo11n-seg.pt")  # use yolov8n-seg.pt if faster inference needed

# Open input video
cap = cv2.VideoCapture("sample.mp4")

# Get video properties
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer (full resolution)
out = cv2.VideoWriter(
    "instance-segmentation-object-tracking.avi",
    cv2.VideoWriter_fourcc(*"MJPG"),
    fps,
    (w, h),
)

# Display scaling for small screen
display_scale = 0.6  # scale down to 60% for 13-inch laptop

# Start time for FPS calculation
prev_time = 0

while True:
    ret, im0 = cap.read()
    if not ret:
        print("✅ Video processing completed.")
        break

    # YOLO segmentation + tracking
    results = model.track(im0, persist=True, verbose=False)

    if results[0].boxes.is_track and results[0].masks is not None:
        masks = results[0].masks.xy                     # segmentation polygons
        boxes = results[0].boxes.xyxy.cpu().numpy()     # bounding boxes
        track_ids = results[0].boxes.id.int().cpu().tolist()  # tracking IDs

        # Annotate each detection
        for mask, box, track_id in zip(masks, boxes, track_ids):
            # Draw segmentation mask
            pts = np.array(mask, dtype=np.int32)
            cv2.fillPoly(im0, [pts], color=colors(int(track_id), True))

            # Draw bounding box
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(im0, (x1, y1), (x2, y2), colors(int(track_id), True), 2)

            # Draw label
            cv2.putText(
                im0,
                f"ID {track_id}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                colors(int(track_id), True),
                2,
            )

    # Calculate FPS
    curr_time = time.time()
    fps_text = f"FPS: {1/(curr_time - prev_time):.2f}" if prev_time != 0 else "FPS: ..."
    prev_time = curr_time
    cv2.putText(im0, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write full-resolution frame to output video
    out.write(im0)

    # Resize frame for display
    display_frame = cv2.resize(im0, (int(w*display_scale), int(h*display_scale)))
    cv2_imshow(display_frame) # Use cv2_imshow

    # Exit on pressing 'q'
    # if cv2.waitKey(1) & 0xFF == ord("q"):
    #     break

# Cleanup
cap.release()
out.release()
# cv2.destroyAllWindows() # Comment out or remove this line